In [1]:
import praw
import random
import re
import time

In [2]:
common_spammy_words = ['udemy', 'course', 'save', 'coupon', 'free', 'discount']

In [3]:
reddit = praw.Reddit(client_id = 'CLIENT_ID',
                     client_secret = 'CLIENT_SECRET',
                     username = 'USERNAME',
                     password = 'PASSWORD',
                     user_agent = 'AGENT')

print(reddit.user.me())

ShifsBot1


In [4]:
def find_spam_by_name(search_query):
    authors = []
    for submission in reddit.subreddit("all").search(search_query, sort="new", limit=11):
        print(submission.title, submission.author, submission.url)
        if submission.author not in authors:
            authors.append(submission.author)
    return authors

In [ ]:
DEBUG_MODE = False
debug_posted = []

while True:
    current_search_query = random.choice(["udemy"])
    spam_content = []
    trashy_users = {}
    smelly_authors = find_spam_by_name(current_search_query)
    for author in smelly_authors:
        user_trashy_urls = [] # Tracks submissions flagged as spam for this user
        sub_count = 0 #Total submissions analyzed for this user.
        dirty_count = 0 #Number of spammy submissions for this user.
        try:
            for sub in reddit.redditor(str(author)).submissions.new(): #for each submission by the author
                submit_links_to = sub.url #The URL of the submission.
                submit_id = sub.id #Unique submission ID.
                submit_subreddit = sub.subreddit #Subreddit where the submission was posted.
                submit_title = sub.title #Title of the submission.
                dirty = False
                for regex in common_spammy_words:
                    #Searches the title against a list of common spam indicators using regex.
                    if re.search(regex, submit_title.lower()):
                        dirty = True
                        junk = [submit_id,submit_title]
                        if junk not in user_trashy_urls:
                            user_trashy_urls.append([submit_id,submit_title,str(author)])
                if dirty:
                    dirty_count += 1
                sub_count += 1
                
            try:
                trashy_score = dirty_count/sub_count
            except: trashy_score = 0.0
            print("User {} trashy score is: {}".format(str(author), round(trashy_score,3)))

            #If the spam score is 50% or higher and the user has posted more than one submission, they are added to trashy_users with their spam score and total submissions.
            if trashy_score >= 0.5 and sub_count > 1:
                trashy_users[str(author)] = [trashy_score,sub_count]

                for trash in user_trashy_urls:
                    spam_content.append(trash)  

        except Exception as e:
            print(str(e))

    for spam in spam_content:
        spam_id = spam[0]
        spam_user = spam[2]
        submission = reddit.submission(id=spam[0])
        created_time = submission.created_utc
        tagged = False

        for comment in submission.comments.list():
            comment_text = comment.body
            if "*Beep boop*" in comment_text:
                print("This submission has already been tagged.")
                tagged = True

        if tagged:
            continue

        if time.time()-created_time <= 86400:
            link = "https://reddit.com"+submission.permalink

            message = """*Beep boop*

I am a bot that sniffs out spammers, and this smells like spam.

At least {}% out of the {} submissions from /u/{} appear to be for Udemy affiliate links. 

Don't let spam take over Reddit! Throw it out!

*Bee bop*""".format(round(trashy_users[spam_user][0]*100,2), trashy_users[spam_user][1], spam_user)
            try:
                if DEBUG_MODE:
                    if link in debug_posted:
                        continue
                    print(f"Would've posted reply to post by {spam_user}: {link}")
                    debug_posted.append(link)
                    continue

                with open("posted_urls.txt","r") as f:
                    already_posted = f.read().split('\n')
                if link not in already_posted:
                    print(message)
                    submission.reply(message)
                    print("We've posted to {} and now we need to sleep for 12 minutes".format(link))
                    with open("posted_urls.txt","a") as f:
                        f.write(link+'\n')
                    time.sleep(12*60)
                    break
            except Exception as e:
                print(str(e))
                time.sleep(12*60)

taskbar misaligned or inappropriate; reboot or restart doesnt help. sivvaaaaaaah https://i.redd.it/gg0mc068z63e1.png
I wanna increase my knowledge in robotics and hopefully turn it into a job nichonichot https://www.reddit.com/r/robotics/comments/1h05iox/i_wanna_increase_my_knowledge_in_robotics_and/
Advance Your Career: 100 Free Courses with Certificates on Udemy  courzeorg /r/Udemy/comments/1h05dfk/advance_your_career_100_free_courses_with/
Advance Your Career: 100 Free Courses with Certificates on Udemy  courzeorg https://www.reddit.com/r/Udemy/comments/1h05dfk/advance_your_career_100_free_courses_with/
Pmpfocus review please. whitehawk04 https://www.reddit.com/r/pmp/comments/1h044o8/pmpfocus_review_please/
Share Your Non-CS Journey: Learning DSA - We Want to Hear Your Story (non-cs major, USA) ThinkingVincent https://www.reddit.com/r/SurveyExchange/comments/1h03n5e/share_your_noncs_journey_learning_dsa_we_want_to/
Share Your Non-CS Journey: Learning DSA - We Want to Hear Your Story